## Combining the CSV of all Participants

In [ ]:
import os
import sys
print (os.getcwd())
sys.path.append('/home/Hanna.Drimalla/ml-autism/scripts')

import pandas as pd
import datetime as dt
import imageio
import numpy as np
import subprocess
import seaborn as sns
import matplotlib.pyplot as plt

import scipy
from scipy import stats
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot
from statsmodels.regression.mixed_linear_model import MixedLMParams
from statsmodels.stats.weightstats import DescrStatsW

import mystats 

import statsmodels.stats.api as sms
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn

from scipy.stats.mstats import zscore
from mpl_toolkits.mplot3d import Axes3D
from sklearn import decomposition
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.preprocessing import StandardScaler
import scipy.cluster.hierarchy as hac
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

import mycharite

import warnings
warnings.simplefilter('ignore')

## Analysis of SSI-Video itself

In [ ]:
study='charite'
VIT=mycharite.load_VIT_actress_AU()

print (len(VIT))
VIT=VIT[VIT.success==1].reset_index(drop=True) #exclude non-sucessfull tracks
print (len(VIT))
print ('all non-successfully tracked frames excluded')

VIT=VIT[VIT.confidence>0.75].reset_index(drop=True)
print ('all frames excluded that were tracked with a lower confidence than 0.75')
print (len(VIT))


#VIT=mycharite.outlier_trials(VIT,['AU06_r', 'AU12_r','AU15_r', 'AU09_r', 'AU04_r'], 'sd')
print (len(VIT))

#a=VIT[VIT.conversation_part>1].groupby('frame').mean()[['AU06_r', 'AU12_r','AU15_r', 'AU09_r', 'AU04_r']]
a=VIT[VIT.conversation_part>1].groupby('conversation_part').mean()[['AU06_c', 'AU12_c','AU15_c', 'AU09_c', 'AU04_c']]

print (a)

plt.plot(a)
#plt.legend(['AU06_c', 'AU12_c', 'AU15_c', 'AU09_c',  'AU04_c'])# da stimmt was mit der LEgende nicht!!
plt.savefig('AU_actress.png')
plt.show()

a=VIT[VIT.conversation_part>1].groupby('conversation_part').std()[['AU06_c', 'AU12_c','AU15_c', 'AU09_c', 'AU04_c']]
print (a)

In [ ]:
var=['AU06_r', 'AU12_r','AU15_r', 'AU09_r', 'AU04_r']
#speaking
print ('joy')
for i in var:
    print (i)

    print (np.mean(VIT[VIT.conversation_part==4][i]))

print ('disgust')
for i in var:
    print (i)
    print (np.mean(VIT[VIT.conversation_part==6][i]))

In [ ]:
var=['AU06_r', 'AU12_r','AU15_r', 'AU09_r', 'AU04_r']
#speaking
print ('joy')
for i in var:
    print (i)

    print (np.mean(VIT[VIT.conversation_part==5][i]))

print ('disgust')
for i in var:
    print (i)
    print (np.mean(VIT[VIT.conversation_part==7][i]))

In [ ]:
#speaking
var=['AU06_c', 'AU12_c']
for i in var:
    print (i)
    speaks=VIT[(VIT.conversation_part==2) | (VIT.conversation_part==4) | (VIT.conversation_part==6)]
    listens=VIT[(VIT.conversation_part==3) | (VIT.conversation_part==5) | (VIT.conversation_part==7)]
    print(np.mean(speaks[i]))
    print(np.mean(listens[i]))
    #mystats.two_ind_sample_tests(speaks, listens, i)

In [ ]:
def what_is():
    for i in set(VIT.conversation_part):
        print (i)
        print (len(VIT[VIT.conversation_part==i])/25)

    print (len(VIT)/25)
    len(onep)/30

    df_vpn=df[df.vpn==4]

    for i in set(df_vpn.conversation):
        print (i)
        print (len(df_vpn[df_vpn.conversation==i])/25)

    print (len(VIT)/25)
    len(df_vpn)/30

# Analysis of Charite Data

### Load dataframe

In [ ]:
VIT=mycharite.load_VIT_actress_AU()
df, df_audio, action_r, action_c, gaze, audio =mycharite.load('charite')
mimicry, new_features=mycharite.calc_mim(df, VIT)
df=pd.merge(df, mimicry, on='vpn')

df=mycharite.aq_charite(df)
df=mycharite.ados_charite(df)
df=mycharite.adapt_times(df)

In [ ]:
#mycharite.check(df)
print ('NT ' + str(sum(df.groupby('vpn').mean().asc==0)))
print ('ASC ' + str(sum(df.groupby('vpn').mean().asc==1)))

In [ ]:
print (len(action_r))

### Korrelation of Diagnosis and AQ

In [ ]:
from scipy import stats
df_aq=df.groupby('vpn').mean().dropna(subset=['asq']).reset_index(drop=True)
print (stats.pointbiserialr(df_aq.asc, df_aq.asq))

### Distribution of AQ in NT and ASC

In [ ]:
df[df.asq>0].groupby('vpn').mean().groupby('asc')['asq'].describe()

### Was the Feature Extraction Successfull and Complete for all Subjects?

In [ ]:
def show_success_vpn(df):
    for i in set(df.vpn):
        print (i)
        print (len(df[df.vpn==i]))      
        print ('sucess')
        print (np.mean(df[df.vpn==i].success))  # Number of sucessfull trackted trials!
        print (np.mean(df[df.vpn==i].success)/len(df[df.vpn==i]))  

#### Unterscheiden sich die Gruppen hinsichtlich, wie gut sie erkannt wurden?

In [ ]:
print (np.mean(df.success))
print (np.std(df.success))

print (np.mean(df.confidence))
print (np.std(df.confidence))
print (np.mean(df.confidence)-3*np.std(df.confidence))
print (np.min(df.groupby('vpn').mean().confidence))

print ('NT')
print (np.mean(df[df.asc==0].groupby('vpn').mean().success))
print (np.std(df[df.asc==0].groupby('vpn').mean().confidence))
print (np.mean(df[df.asc==0].groupby('vpn').mean().confidence))

print ('ASC')
print (np.mean(df[df.asc==1].groupby('vpn').mean().success))
print (np.mean(df[df.asc==1].groupby('vpn').mean().confidence))
print (np.std(df[df.asc==1].groupby('vpn').mean().confidence))

print (scipy.stats.ttest_ind(df[df.asc==0].groupby('vpn').mean().confidence, 
                             df[df.asc==1].groupby('vpn').mean().confidence))

### Exclude non-confident trials and Outliers

In [ ]:
df=mycharite.exclude_outlier(df)

In [ ]:
def outlier_detection(df):
    var=['AU06_r', 'AU12_r', 'AU04_r']
    for vpn in set(df.vpn):
        df.loc[(df.vpn==vpn), :]=mycharite.outlier_trials(df.loc[(df.vpn==vpn), :], var, 'iq')
    return df

# this excludes a lot of frames - thus might not be useful as non-normally distributed

### Descriptives

In [ ]:
face=['AU04_r', 'AU06_r', 'AU09_r', 'AU12_r']
new_features=new_features+face
for i in new_features:
    print (i)
    print (df.groupby('vpn').mean().groupby('asc')[i].describe())

###  Smoothing the Signal

In [ ]:
def smooth(df):
    round_df=df
    for i in set(df.vpn):
        for au in action_r:
            round_df.loc[round_df.vpn==i, au]=round_df.loc[round_df.vpn==i, au].rolling(30, win_type='triang').mean()
    return round_df

# 1) Mimicry of Joy and Disgust 

First, we checked whether the participants showed a similar feeling as the actress in the conversation part of disgust and joy. We compared the facial expression with the facial expression of the neutral part.

In [ ]:
df['AU_joy']=(df.AU12_r + df.AU06_r)/2
df['AU_disgust']=df.AU04_r

In [ ]:
df_neutral=df[((df.conversation=='neutral_speaker')|(df.conversation=='neutral_participant'))].groupby(['vpn']).mean().reset_index(drop=True)

## Joy

In [ ]:
#compared to neutral
df_joy=df[((df.conversation=='joy_speaker') |(df.conversation=='joy_participant'))].groupby(['vpn']).mean().reset_index(drop=True)
print ('AU06_C')
mystats.two_dep_sample_tests(df_neutral, df_joy, 'AU06_c')
print ('AU12_C')
mystats.two_dep_sample_tests(df_neutral, df_joy, 'AU12_c')

#compared to neutral
df_joy=df[((df.conversation=='joy_speaker') |(df.conversation=='joy_participant'))].groupby(['vpn']).mean().reset_index(drop=True)
print ('AU06_r')
mystats.two_dep_sample_tests(df_neutral, df_joy, 'AU06_r')
print ('AU12_r')
mystats.two_dep_sample_tests(df_neutral, df_joy, 'AU12_r')

## Disgust

In [ ]:
#compared to neutral
df_disgust=df[((df.conversation=='disgust_speaker')|(df.conversation=='disgust_participant'))].groupby(['vpn']).mean().reset_index(drop=True)
#my.two_ind_sample_tests(df_disgust, df_neutral, 'AU04_r')

mystats.two_dep_sample_tests(df_neutral, df_disgust, 'AU04_c')
mystats.two_dep_sample_tests(df_neutral, df_disgust, 'AU04_r')

## 2) Statistical Comparing the AU of ASD and NT

In [ ]:
def group_comparison(df):
    print ('joy')
    df_joy=df[((df.conversation=='joy_speaker') |(df.conversation=='joy_participant'))].groupby(['vpn']).mean().reset_index(drop=True)

    df_nt=df_joy[df_joy.asc==0]
    df_asc=df_joy[df_joy.asc==1]

    print ('AU_06c')
    mystats.two_ind_sample_tests(df_nt, df_asc, 'AU06_c')

    print ('AU_06r')
    mystats.two_ind_sample_tests(df_nt, df_asc, 'AU06_r')

    print ('AU12_c')
    U,p= scipy.stats.mannwhitneyu(df_nt['AU12_c'], df_asc['AU12_c'])
    mystats.two_ind_sample_tests(df_nt, df_asc, 'AU12_c')
    n1=len(df_nt['AU12_c'])
    n2=len(df_asc['AU12_c'])
    effectsize=1-(2*U/(n1*n2))
    print (effectsize)
    print ('AU_12r')
    mystats.two_ind_sample_tests(df_nt, df_asc, 'AU12_r')


    print ('disgust')
    df_disgust=df[((df.conversation=='disgust_speaker')|(df.conversation=='disgust_participant'))].groupby(['vpn']).mean().reset_index(drop=True)

    df_nt=df_disgust[df_disgust.asc==0]
    df_asc=df_disgust[df_disgust.asc==1]
    print('AU04_c')
    mystats.two_ind_sample_tests(df_nt, df_asc, 'AU04_c')
    print('AU04_r')
    mystats.two_ind_sample_tests(df_nt, df_asc, 'AU04_r')


    print ('complete conversation')
    df=df[df.conversation!='intro']

    df_nt=df[df.asc==0].groupby(['vpn']).mean().reset_index(drop=True)
    df_asc=df[df.asc==1].groupby(['vpn']).mean().reset_index(drop=True)
    print ('AU12_C')
    mystats.two_ind_sample_tests(df_nt, df_asc, 'AU12_c')
    print ('AU12_r')
    mystats.two_ind_sample_tests(df_nt, df_asc, 'AU12_r')
    print ('AU06_C')
    mystats.two_ind_sample_tests(df_nt, df_asc, 'AU06_c')
    print ('AU06_r')
    mystats.two_ind_sample_tests(df_nt, df_asc, 'AU06_r')

In [ ]:
group_comparison(df)

In [ ]:
def mimicry_group_conversation(df, features):
    print ('complete conversation')
    df=df[df.conversation!='intro']
    
    df_nt=df[df.asc==0].groupby(['vpn']).mean().reset_index(drop=True)
    df_asc=df[df.asc==1].groupby(['vpn']).mean().reset_index(drop=True)
    for fea in features:

        print (fea)
        mystats.two_ind_sample_tests(df_nt[df_nt[fea]>0], df_asc[df_asc[fea]>0], fea)
    

In [ ]:
features=['correlation_AU06_c', 'correlation_AU12_c', 'correlation_AU15_r',
   'correlation_AU09_r', 'correlation_AU04_r', 'correlation_AU15_c',
   'correlation_AU09_c', 'correlation_AU04_c',
         'correlation_gaze_angle_x', 'correlation_gaze_angle_y']

mimicry_group_conversation(df, features)

In [ ]:
mimicry_group_conversation(df, features)

In [ ]:
#FEMALE
def gender_based_analysis (df):
    print ('female')
    group_comparison(df[df.sex==1])
    print ('male')
    group_comparison(df[df.sex==2])

In [ ]:
def time_plot(df, var, title):
    plt.plot(np.array(df[df.asc==1].reset_index(drop=True).groupby(['counter']).mean()[var]), color='green')
    plt.plot(np.array(df[df.asc==0].reset_index(drop=True).groupby(['counter']).mean()[var]), color='blue')
    #scale!!!
    plt.title(str(title))
    plt.show()
  
   
for con in np.unique(df.conversation):
    time_plot(df[df.conversation==con], 'AU12_r', con) #sc

In [ ]:
df['proband']=(df.conversation=='joy_participant')|(df.conversation=='disgust_participant')|(df.conversation=='neutral_participant')
df['actress']=(df.conversation=='joy_speaker')|(df.conversation=='disgust_speaker')|(df.conversation=='neutral_speaker')
df.loc[(df.conversation!='intro'), "whospeaks"] = df[['proband', 'actress']].idxmax(axis=1) 

In [ ]:
df.columns
from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()
df['conversationpart'] = LE.fit_transform(df['conversation'])

In [ ]:
print ('AU12')
data=df[df.sex==1].groupby(['vpn', 'conversationpart']).mean().reset_index()
model = sm.MixedLM.from_formula('AU12_c' + " ~ asc*conversationpart", data, groups=data['vpn']) #, re_formula="~conversationpart")
mdf = model.fit()
print(mdf.summary())

In [ ]:
plt.hist(df.groupby(['vpn']).mean().reset_index().AU12_r)
plt.show()

## Mimicry-Analysis

In [ ]:
new_features=['AU06_r', 'AU12_r']
for i in new_features:
    print (i)
    plt.hist(df[df.asc==0].groupby(['vpn']).mean().reset_index()[i], color='blue')
    print (np.mean(df[df.asc==0].groupby(['vpn']).mean().reset_index()[i]))
    plt.hist(df[df.asc==1].groupby(['vpn']).mean().reset_index()[i], color='red')
    print (np.mean(df[df.asc==1].groupby(['vpn']).mean().reset_index()[i]))
    plt.show()
    print (scipy.stats.ttest_ind((df[df.asc==0].groupby(['vpn']).mean().reset_index()[i]), 
           (df[df.asc==1].groupby(['vpn']).mean().reset_index()[i])))

In [ ]:
print ('AU12')
data=df.groupby(['vpn', 'conversationpart']).mean().reset_index()
model = sm.MixedLM.from_formula('AU12_c' + " ~ asc+sex", data, groups=data['vpn']) #, re_formula="~conversationpart")
mdf = model.fit()
print(mdf.summary())

print ('AU12')
data=df.groupby(['vpn', 'conversationpart']).mean().reset_index()
model = sm.MixedLM.from_formula('AU12_r' + " ~ asc", data, groups=data['vpn']) #, re_formula="~conversationpart")
mdf = model.fit()
print(mdf.summary())

In [ ]:
print ('AU12')
data=df[df.sex==2].groupby(['vpn', 'conversationpart']).mean().reset_index()
model = sm.MixedLM.from_formula('AU12_c' + " ~ asc*conversationpart", data, groups=data['vpn']) #, re_formula="~conversationpart")
mdf = model.fit()
print(mdf.summary())

In [ ]:
print ('AU12')
data=df[df.sex==1].groupby(['vpn', 'conversationpart']).mean().reset_index()
model = sm.MixedLM.from_formula('AU12_r' + " ~ asc*conversationpart", data, groups=data['vpn']) #, re_formula="~conversationpart")
mdf = model.fit()
print(mdf.summary())

In [ ]:
print ('AU12')
data=df[df.sex==2].groupby(['vpn', 'conversationpart']).mean().reset_index()
model = sm.MixedLM.from_formula('AU12_r' + " ~ asc*conversationpart", data, groups=data['vpn']) #, re_formula="~conversationpart")
mdf = model.fit()
print(mdf.summary())

In [ ]:
print ('AU12')
data=df.groupby(['vpn', 'conversationpart']).mean().reset_index()
model = sm.MixedLM.from_formula('AU06_r' + " ~ asc*conversationpart", data, groups=data['vpn']) #, re_formula="~conversationpart")
mdf = model.fit()
print(mdf.summary())

In [ ]:
def AUboxplot_ASC_NT(df):
    relevant_AU=['AU06_r', 'AU12_r']#, 'AU09_r','AU15_r', 'AU04_r']
    for i in relevant_AU:
        plt.figure(figsize=(18, 8)) 
        fig=sns.violinplot(x='conversation', y=i, hue='asc', data=df)
        plt.yticks(fontsize=16)    
        plt.xticks(fontsize=16)  
        plt.xlabel('Conversation Parts', fontsize=18) 
        plt.ylabel(('Intensity of Action Unit ' + str(i)), fontsize=18)
        plt.legend(title='ASC', loc = "upper left", fontsize=16)
        plt.savefig('AU_'+ str(i) + '_.png')
        #print (df[df.asc==0].reset_index(drop=True).groupby(['vpn', 'conversation']).mean()[i])
        plt.show()
AUboxplot_ASC_NT(df)

In [ ]:
relevant_AU=['AU06_r', 'AU12_r']#, 'AU09_r','AU15_r', 'AU04_r']
for i in relevant_AU:
    plt.figure(figsize=(18, 8)) 
    fig=sns.violinplot(x='asc', y=i, data=df)
    plt.yticks(fontsize=16)    
    plt.xticks(fontsize=16)  
    plt.xlabel('Conversation Parts', fontsize=18) 
    plt.ylabel(('Intensity of Action Unit ' + str(i)), fontsize=18)
    plt.legend(title='ASC', loc = "upper left", fontsize=16)
    plt.savefig('AU_'+ str(i) + '_.png')
    #print (df[df.asc==0].reset_index(drop=True).groupby(['vpn', 'conversation']).mean()[i])
    plt.show()


In [ ]:
relevant_AU=['AU06_r', 'AU12_r']#, 'AU09_r','AU15_r', 'AU04_r']
sns.set_style("whitegrid")
sns.set_context("talk")
for i in relevant_AU:
    plt.figure(figsize=(6, 8)) 
    fig=sns.boxplot(x='asc', y=i, data=df, whis=1.5, palette=sns.color_palette("colorblind"))
    plt.yticks(fontsize=16)    
    plt.xticks(fontsize=16)  
    plt.xlabel('Group', fontsize=18) 
    plt.ylabel(('Intensity of Action Unit ' + str(i)), fontsize=18)
    #plt.legend(title='ASC', loc = "upper left", fontsize=16)
    plt.savefig('AU_'+ str(i) + '_.png')
    plt.savefig('AU_'+ str(i) + '_.pdf')
    #print (df[df.asc==0].reset_index(drop=True).groupby(['vpn', 'conversation']).mean()[i])
    plt.show()

In [ ]:
# Association with Markers

In [ ]:
demog=pd.read_csv('./charite/demo_data_charite.csv', encoding='latin-1')

In [ ]:
def associations(ados):
    var=['AU12_r', 'AU12_c', 'AU06_r', 'AU06_c']


    for v in var:
        print (v)
        print('ados-social')
        a=ados[(ados.asc==1)&(ados.conversation!='intro')].groupby('vpn').mean()[['ados_social', v]].dropna()
        mystats.correlation(a.ados_social, a[v])
        print('aq')
        a=ados[(ados.asc==1)&(ados.conversation!='intro')].groupby('vpn').mean()[['asq', v]].dropna()
        mystats.correlation(a.asq, a[v])
        print('ados')
        a=ados[(ados.asc==0)&(ados.conversation!='intro')].groupby('vpn').mean()[['asq', v]].dropna()
        mystats.correlation(a.asq, a[v])

In [ ]:
associations(df[df.sex==1])

In [ ]:
associations(df[df.sex==2])

In [ ]:
stats.ttest_ind(df[(df.asd==11) & (df.sex==1)].groupby('vpn').mean().asq.dropna(),
                df[(df.asd==11) & (df.sex==2)].groupby('vpn').mean().asq.dropna())

In [ ]:
stats.ttest_ind(df[(df.asd==11) & (df.sex==1)].groupby('vpn').mean().ados_total.dropna(), 
                df[(df.asd==11) & (df.sex==2)].groupby('vpn').mean().ados_total.dropna())

In [ ]:
print (np.mean(df[(df.asd==11) & (df.sex==1)].groupby('vpn').mean().ados_total.dropna()))
print (np.mean(df[(df.asd==11) & (df.sex==2)].groupby('vpn').mean().ados_total.dropna()))

stats.ttest_ind(df[(df.asd==11) & (df.sex==1)].groupby('vpn').mean().ados_total.dropna(), 
       df[(df.asd==11) & (df.sex==2)].groupby('vpn').mean().ados_total.dropna())

In [ ]:
print (np.mean(df[(df.asd==11) & (df.sex==1)].groupby('vpn').mean().asq.dropna()))
print (np.mean(df[(df.asd==11) & (df.sex==2)].groupby('vpn').mean().asq.dropna()))


stats.ttest_ind(df[(df.asd==11) & (df.sex==1)].groupby('vpn').mean().asq.dropna(), 
       df[(df.asd==11) & (df.sex==2)].groupby('vpn').mean().asq.dropna())